In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum,when,round

spark = SparkSession \
    .builder \
    .appName("leetcode-problems") \
    .master("local[*]") \
    .config("spark.jars", "/Users/sahiltest/Downloads/mysql-connector-java-8.0.29.jar") \
    .getOrCreate()

titanic = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "titanic") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

titanic.show()

25/06/09 18:07:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/09 18:07:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/09 18:07:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-----+-----+--------+
|passengerid|survived|pclass|                name|   sex|  age|sibsp|parch|          ticket| fare|cabin|embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.00|    1|    0|       A/5 21171| 7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.00|    1|    0|        PC 17599|71.28|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.00|    0|    0|STON/O2. 3101282| 7.92| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.00|    1|    0|          113803|53.10| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.00|    0|    0|          373450| 8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male| null|    0|    0|          330

In [12]:
first = titanic\
    .filter(col("pclass")==1)\
    .groupBy(col("survived"))\
    .agg(count("passengerid").alias("first_class"))

second = titanic\
    .filter(col("pclass")==2)\
    .groupBy(col("survived"))\
    .agg(count("passengerid").alias("second_class"))

third = titanic\
    .filter(col("pclass")==3)\
    .groupBy(col("survived"))\
    .agg(count("passengerid").alias("third_class"))

df=first\
    .join(second,first.survived == second.survived,"left")\
    .select(first.survived,first.first_class,second.second_class)

df\
    .join(third,df.survived == third.survived,"left")\
    .select(df.survived,df.first_class,df.second_class,third.third_class)\
    .show()

+--------+-----------+------------+-----------+
|survived|first_class|second_class|third_class|
+--------+-----------+------------+-----------+
|       1|          2|           1|          2|
|       0|          1|        null|          4|
+--------+-----------+------------+-----------+

